In [1]:
import json
import pandas as pd

In [7]:
#HELPER FUNCTIONS
def get_addr_data(json_dataset):
    """
    # info on available address fields ( and how are they used )
    # export addr data to pandas-dataset [ element-id, addr-field, addr-field-val ]
    #   run analysis of the dataset
    """
    data_header = ['node_id', 'node_type', 'tag', 'val']
    raw_data = []
    for json_node in json_dataset:
        if 'address' in json_node:
            for tag in json_node['address']:
                raw_data.append([
                    json_node['id'],
                    json_node['type'],
                    tag,
                    json_node['address'][tag]
                ])
        else:
            raw_data.append([
                json_node['id'],
                json_node['type'],
                None,
                None
            ])
    return pd.DataFrame(raw_data, columns=data_header)




In [8]:
#CONST DEFINITIONS
DATA_SOURCE='..\sample-10000-redwood-city-ca.osm.json'

#LOADING DATA
parsed_json_data = []
for line in open(DATA_SOURCE, 'r'):
    parsed_json_data.append(json.loads(line))
    
data_df = get_addr_data(parsed_json_data)

In [9]:
print data_df[0]

KeyError: 0